In [1]:
#import sys
#!{sys.executable} -m pip install --upgrade "scikit-learn>=1.6.1,<1.7" category_encoders shap lyft_fugue

In [27]:
#usual packages
import sklearn
import numpy as np
from IPython.display import IFrame
from sklearn.model_selection import GroupKFold
from datetime import date
import pandas as pd
from pandas import DataFrame
import subprocess
import sys

#lyft packages
import lyft_data_toolkit.db.write.df
from lyft_data_toolkit.db.read import Compute
from lyft_data_toolkit.db.read import Query
from lyft_data_toolkit.db.read.df import to_dataframe  
from lyft_data_toolkit.v1_to_v3 import hive
from lyft_data_toolkit.v1_to_v3 import presto
from lyft_data_toolkit.v1_to_v3 import spark
from lyft_data_toolkit.v1_to_v3 import trino

#other packages
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

### 1. Run/change parameters

In [30]:
launch_date = "2025-08-04"

In [3]:
# import os
# print(os.getcwd())

/mnt/custom-file-systems/efs/fs-04bf86d02daf87e14/BT


In [25]:
lift_lookup = pd.read_csv("weekly_lift_data.csv")
### insert in new on demand data 
lift_lookup = pd.concat([
    lift_lookup,
    pd.DataFrame([{
        "week_num": 40, #######
        "year_num": 2025, ###
        "starting_week_date": "2025-09-29", #######
        "pre_existing_rider_lift": 0.0227, #########
        "new_rider_pp_lift": 1.833 #######
    }])
], ignore_index=True)

lift_lookup = lift_lookup.sort_values("starting_week_date").drop_duplicates(ignore_index=True).reset_index(drop=True)
lift_lookup
lift_lookup.to_csv("weekly_lift_data.csv",index =False)

In [31]:
lift_lookup

,week_num,year_num,starting_week_date,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0.0227,1.8330
1,33,2025,2025-08-11,0.0227,1.8330
2,34,2025,2025-08-18,0.0227,1.8330
3,35,2025,2025-08-25,0.0227,1.8330
4,36,2025,2025-09-01,0.0227,1.8330
5,37,2025,2025-09-08,0.0227,1.8330
6,38,2025,2025-09-15,0.0227,1.8330
7,39,2025,2025-09-22,0.0227,1.8330
8,40,2025,2025-09-29,0.0227,1.8330
9,41,2025,2025-10-06,0.0130,1.2870


In [28]:
from datetime import datetime, timedelta

today = datetime.today().date()

# Get previous week's Monday
report_start_dt = today - timedelta(days=today.weekday() + 7)
# Get previous week's Sunday
report_end_dt = report_start_dt + timedelta(days=6)

# Output as strings
report_start_dt = report_start_dt.strftime('%Y-%m-%d')
report_end_dt = report_end_dt.strftime('%Y-%m-%d')

print(report_start_dt, report_end_dt)

2025-11-10 2025-11-16


### 2. Run Prediction

In [32]:

SQL ="""
--higherlimit
WITH pre_final_table AS (
SELECT DISTINCT 
       sub1.rider_lyft_id,
       sub1.business_program_user_id,
       sub1.business_program_type,
       sub1.business_program_type_cleaned,
       sub1.lyft_signed_up_at,
       sub1.bp_created_at,
       sub1.bp_activated_at,
       sub1.user_type,
       sub1.sub_user_type,
       sub2.first_ride_finished_at
  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1
  LEFT JOIN coco.dim_user sub2 
    ON sub1.rider_lyft_id = sub2.user_lyft_id
 WHERE sub1.is_business_ride = TRUE 
) 

,pre_final_table_v2 AS (
SELECT rider_lyft_id,
       business_program_user_id,
       business_program_type,
       business_program_type_cleaned,
       lyft_signed_up_at,
       bp_created_at,
       bp_activated_at,
       user_type,
       sub_user_type,
       first_ride_finished_at,
       ROW_NUMBER() OVER(PARTITION BY rider_lyft_id ORDER BY bp_activated_at DESC) rnk 
  FROM pre_final_table 
 WHERE TRUE
)

SELECT DATE_TRUNC('week',CAST(bp_activated_at AS DATE)) week,
       business_program_type_cleaned,
       CASE WHEN DATE_DIFF('day',first_ride_finished_at,bp_activated_at) > 0 THEN 'PP' ELSE 'NP' END AS activation_type, -- proxy for NP/PP
       COUNT(DISTINCT rider_lyft_id) count_activations
  FROM pre_final_table_v2
 WHERE TRUE 
   AND rnk = 1
 GROUP BY 1,2,3
"""  
df = pd.DataFrame()
df = trino.query(SQL, scheduled=True)

{"ts": "2025-11-18T00:04:44.238Z", "uuid": "b0fb6bda-772a-4dd7-8032-f9ed3759df6b", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}
/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:347: UserWarning: Attempting to establish a new TCP stats connection
  warnings.warn("Attempting to establish a new TCP stats connection")


Created Mozart command
{'mozart_id': 569115131, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569115131', 'query': "--higherlimit\nWITH pre_final_table AS (\nSELECT DISTINCT \n       sub1.rider_lyft_id,\n       sub1.business_program_user_id,\n       sub1.business_program_type,\n       sub1.business_program_type_cleaned,\n       sub1.lyft_signed_up_at,\n       sub1.bp_created_at,\n       sub1.bp_activated_at,\n       sub1.user_type,\n       sub1.sub_user_type,\n       sub2.first_ride_finished_at\n  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1\n  LEFT JOIN coco.dim_user sub2 \n    ON sub1.rider_lyft_id = sub2.user_lyft_id\n WHERE sub1.is_business_ride = TRUE \n) \n\n,pre_final_table_v2 AS (\nSELECT rider_lyft_id,\n       business_program_user_id,\n       business_program_type,\n       business_program_type_cleaned,\n       lyft_signed_up_at,\n       bp_created_at,\n       bp_activated_at,\n       user_type,\n       sub_us

In [33]:
df.columns

Index(['week', 'business_program_type_cleaned', 'activation_type',
       'count_activations'],
      dtype='object')

In [34]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()

print(df_pivot)

business_program_type_cleaned       week activation_type  managed  organic
0                             2021-08-09              NP       11        1
1                             2021-08-09              PP      931      851
2                             2021-08-16              NP       39        4
3                             2021-08-16              PP     1968     1724
4                             2021-08-23              NP       16        3
..                                   ...             ...      ...      ...
441                           2025-10-27              PP     1415     3985
442                           2025-11-03              NP      188      141
443                           2025-11-03              PP     1477     4633
444                           2025-11-10              NP      134       77
445                           2025-11-10              PP     1129     3554

[446 rows x 4 columns]


In [35]:
np_df = df_pivot[df_pivot.activation_type == "NP"]
pp_df = df_pivot[df_pivot.activation_type == "PP"]

In [36]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from joblib import load

# launch_date = "2025-08-04"

np_df = df_pivot[df_pivot["activation_type"] == "NP"].copy()
pp_df = df_pivot[df_pivot["activation_type"] == "PP"].copy()

def add_lags(df):
    df = df.sort_values("week").reset_index(drop=True)
    df["organic_lag1"] = df["organic"].shift(1).fillna(method='bfill')
    df["organic_lag2"] = df["organic"].shift(2).fillna(method='bfill')
    return df

np_df = add_lags(np_df)
pp_df = add_lags(pp_df)

def predict_post_rewards(df, model_file, use_lag2=True):
    post_df = df[df["week"] >= launch_date].copy()
    if post_df.empty:
        print(f"No post-rewards data for {model_file}")
        return None
    
    #features = ["organic", "organic_lag1"]
    #if use_lag2:
        #features.append("organic_lag2")
    
    X_post = post_df["organic"]
    
    model = load(model_file)
    
    post_df["predicted_managed"] = model.predict(
        start=len(df[df["week"] < launch_date]),
        end=len(df)-1,
        exog=X_post
    )
    
    return post_df[["week", "managed", "predicted_managed"]]

np_post_preds = predict_post_rewards(np_df, "arimax_model_np.pkl", use_lag2=False)
pp_post_preds = predict_post_rewards(pp_df, "arimax_model_pp.pkl", use_lag2=True)

########## newly added nov 17, 25
np_post_preds['predicted_managed'] = np_post_preds['predicted_managed'].round(0)
pp_post_preds['predicted_managed'] = pp_post_preds['predicted_managed'].round(0)

if np_post_preds is not None:
    np_post_preds.to_csv("np_post_rewards_predictions.csv", index=False)
    print("NP post-rewards predictions exported to np_post_rewards_predictions.csv")

if pp_post_preds is not None:
    pp_post_preds.to_csv("pp_post_rewards_predictions.csv", index=False)
    print("PP post-rewards predictions exported to pp_post_rewards_predictions.csv")

print("NP Post-Rewards Predictions:")
print(np_post_preds)

print("PP Post-Rewards Predictions:")
print(pp_post_preds)


NP post-rewards predictions exported to np_post_rewards_predictions.csv
PP post-rewards predictions exported to pp_post_rewards_predictions.csv
NP Post-Rewards Predictions:
business_program_type_cleaned       week  managed  predicted_managed
208                           2025-08-04      134              133.0
209                           2025-08-11      120              143.0
210                           2025-08-18      131              142.0
211                           2025-08-25       92              121.0
212                           2025-09-01      109              111.0
213                           2025-09-08      153              161.0
214                           2025-09-15      172              161.0
215                           2025-09-22      151              144.0
216                           2025-09-29      170              126.0
217                           2025-10-06      195              152.0
218                           2025-10-13      146              168.0

### 3. calculate additional acitivation percentage - get incremental part into dataframe

In [37]:
np_preds = np_post_preds.reset_index(drop= True)
pp_preds = pp_post_preds.reset_index(drop = True)

In [42]:
np_preds['addl_activations'] = ((np_preds['managed'] - np_preds['predicted_managed']).clip(lower=0)).astype(int)
pp_preds['addl_activations'] = ((pp_preds['managed'] - pp_preds['predicted_managed']).clip(lower=0)).astype(int)

In [43]:
np_preds['perc_addl_activations'] = np_preds['addl_activations']/np_preds['managed']
pp_preds['perc_addl_activations'] = pp_preds['addl_activations']/pp_preds['managed']

In [44]:
np_preds


business_program_type_cleaned,week,managed,predicted_managed,addl_activations,perc_addl_activations
0,2025-08-04,134,133.0,1,0.007463
1,2025-08-11,120,143.0,0,0.000000
2,2025-08-18,131,142.0,0,0.000000
3,2025-08-25,92,121.0,0,0.000000
4,2025-09-01,109,111.0,0,0.000000
5,2025-09-08,153,161.0,0,0.000000
6,2025-09-15,172,161.0,11,0.063953
7,2025-09-22,151,144.0,7,0.046358
8,2025-09-29,170,126.0,44,0.258824
9,2025-10-06,195,152.0,43,0.220513


### 5. to build pre existing, new user pp, np rides table

#### 5.1 get pre existing data

In [45]:
pre_existing_sql  = f"""
-- higherlimit
SELECT week(requested_at) as week_num, year(requested_at) as year_num, 
DATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, 
date_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,
'pre_existing' as user_type,
count(distinct ride_id) as num_rides_pre_existing,
sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing
FROM shuninga.causal_analysis_bp_raw_data_step_4 
 WHERE user_type = 'pre-existing-user'
   AND most_used_business_program_type_cleaned = 'managed' 
  and date(requested_at) between date'{launch_date}' and date'{report_end_dt}'
  group by 1,2,3,4,5
  order by 1,2
  """ 
pre_exisiting_rides = pd.DataFrame()
pre_exisiting_rides = trino.query(pre_existing_sql, scheduled= True)

{"ts": "2025-11-18T00:08:32.884Z", "uuid": "4b064560-28e9-47b7-a0f5-d45a8cc2fa10", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 569116871, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569116871', 'query': "-- higherlimit\nSELECT week(requested_at) as week_num, year(requested_at) as year_num, \nDATE_TRUNC('week',CAST(requested_at AS DATE)) starting_week_date, \ndate_diff('week',date'2025-08-04', requested_at) as post_reward_week_num,\n'pre_existing' as user_type,\ncount(distinct ride_id) as num_rides_pre_existing,\nsum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', requested_at) asc) as cum_rides_pre_existing\nFROM shuninga.causal_analysis_bp_raw_data_step_4 \n WHERE user_type = 'pre-existing-user'\n   AND most_used_business_program_type_cleaned = 'managed' \n  and date(requested_at) between date'2025-08-04' and date'2025-11-16'\n  group by 1,2,3,4,5\n  order by 1,2"}
s3://mozartgateway-production-iad/mz/cmd/results/569116871/


In [47]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   week_num                15 non-null     int64         
 1   year_num                15 non-null     int64         
 2   starting_week_date      15 non-null     datetime64[ns]
 3   post_reward_week_num    15 non-null     int64         
 4   user_type               15 non-null     object        
 5   num_rides_pre_existing  15 non-null     int64         
 6   cum_rides_pre_existing  15 non-null     int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 968.0+ bytes


In [48]:
lift_lookup["starting_week_date"] = pd.to_datetime(lift_lookup["starting_week_date"])
pre_exisiting_rides = pd.merge(
    pre_exisiting_rides,
    lift_lookup[["starting_week_date", "pre_existing_rider_lift"]],
    on="starting_week_date",
    how="left" 
)
pre_exisiting_rides.drop_duplicates(ignore_index=True)

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift
0,32,2025,2025-08-04,0,pre_existing,71479,71479,0.0227
1,33,2025,2025-08-11,1,pre_existing,70331,141810,0.0227
2,34,2025,2025-08-18,2,pre_existing,70152,211962,0.0227
3,35,2025,2025-08-25,3,pre_existing,67829,279791,0.0227
4,36,2025,2025-09-01,4,pre_existing,64151,343942,0.0227
5,37,2025,2025-09-08,5,pre_existing,78855,422797,0.0227
6,38,2025,2025-09-15,6,pre_existing,81989,504786,0.0227
7,39,2025,2025-09-22,7,pre_existing,77048,581834,0.0227
8,40,2025,2025-09-29,8,pre_existing,74461,656295,0.0227
9,41,2025,2025-10-06,9,pre_existing,79833,736128,0.0130


In [49]:
pre_exisiting_rides['pre_existing_incr_rides'] =  (1-1/(1+pre_exisiting_rides['pre_existing_rider_lift']))*pre_exisiting_rides['num_rides_pre_existing']
pre_exisiting_rides['pre_existing_incr_rides'] = pre_exisiting_rides['pre_existing_incr_rides'].round(0) #######
pre_exisiting_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides
0,32,2025,2025-08-04,0,pre_existing,71479,71479,0.0227,1587.0
1,33,2025,2025-08-11,1,pre_existing,70331,141810,0.0227,1561.0
2,34,2025,2025-08-18,2,pre_existing,70152,211962,0.0227,1557.0
3,35,2025,2025-08-25,3,pre_existing,67829,279791,0.0227,1506.0
4,36,2025,2025-09-01,4,pre_existing,64151,343942,0.0227,1424.0
5,37,2025,2025-09-08,5,pre_existing,78855,422797,0.0227,1750.0
6,38,2025,2025-09-15,6,pre_existing,81989,504786,0.0227,1820.0
7,39,2025,2025-09-22,7,pre_existing,77048,581834,0.0227,1710.0
8,40,2025,2025-09-29,8,pre_existing,74461,656295,0.0227,1653.0
9,41,2025,2025-10-06,9,pre_existing,79833,736128,0.0130,1025.0


#### 5.2 get np and pp data

In [50]:
########## pp new user 
# np_preds 
# pp_preds
np_preds

business_program_type_cleaned,week,managed,predicted_managed,addl_activations,perc_addl_activations
0,2025-08-04,134,133.0,1,0.007463
1,2025-08-11,120,143.0,0,0.000000
2,2025-08-18,131,142.0,0,0.000000
3,2025-08-25,92,121.0,0,0.000000
4,2025-09-01,109,111.0,0,0.000000
5,2025-09-08,153,161.0,0,0.000000
6,2025-09-15,172,161.0,11,0.063953
7,2025-09-22,151,144.0,7,0.046358
8,2025-09-29,170,126.0,44,0.258824
9,2025-10-06,195,152.0,43,0.220513


In [51]:
new_user_pp_sql = f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user pp' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_pp,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-PP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_pp_rides = pd.DataFrame()
new_pp_rides = trino.query(new_user_pp_sql,scheduled =True)
new_pp_rides

{"ts": "2025-11-18T00:08:53.866Z", "uuid": "2171e02c-f155-4627-bc64-b07b9ef0d7d5", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 569117093, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569117093', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2025-11-16', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user pp' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_pp,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_pp\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-PP'\n  AND d.most_used_business_program_type_cl

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp
0,32,2025,2025-08-04,0,new user pp,2506,2506
1,33,2025,2025-08-11,1,new user pp,3119,5625
2,34,2025,2025-08-18,2,new user pp,3779,9404
3,35,2025,2025-08-25,3,new user pp,4116,13520
4,36,2025,2025-09-01,4,new user pp,4043,17563
5,37,2025,2025-09-08,5,new user pp,6330,23893
6,38,2025,2025-09-15,6,new user pp,7771,31664
7,39,2025,2025-09-22,7,new user pp,7516,39180
8,40,2025,2025-09-29,8,new user pp,7682,46862
9,41,2025,2025-10-06,9,new user pp,9508,56370


In [52]:
pp_preds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   week                   15 non-null     datetime64[ns]
 1   managed                15 non-null     int64         
 2   predicted_managed      15 non-null     float64       
 3   addl_activations       15 non-null     int64         
 4   perc_addl_activations  15 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 728.0 bytes


In [53]:
new_pp_rides = pd.merge(new_pp_rides,
                        pp_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 

new_pp_rides = pd.merge(new_pp_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift','new_rider_pp_lift']],
                        on = 'starting_week_date',
                        how = 'left')
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift
0,32,2025,2025-08-04,0,new user pp,2506,2506,2025-08-04,0.000000,0.0227,1.8330
1,33,2025,2025-08-11,1,new user pp,3119,5625,2025-08-11,0.000000,0.0227,1.8330
2,34,2025,2025-08-18,2,new user pp,3779,9404,2025-08-18,0.053942,0.0227,1.8330
3,35,2025,2025-08-25,3,new user pp,4116,13520,2025-08-25,0.000000,0.0227,1.8330
4,36,2025,2025-09-01,4,new user pp,4043,17563,2025-09-01,0.000000,0.0227,1.8330
5,37,2025,2025-09-08,5,new user pp,6330,23893,2025-09-08,0.094709,0.0227,1.8330
6,38,2025,2025-09-15,6,new user pp,7771,31664,2025-09-15,0.116335,0.0227,1.8330
7,39,2025,2025-09-22,7,new user pp,7516,39180,2025-09-22,0.079330,0.0227,1.8330
8,40,2025,2025-09-29,8,new user pp,7682,46862,2025-09-29,0.093079,0.0227,1.8330
9,41,2025,2025-10-06,9,new user pp,9508,56370,2025-10-06,0.169360,0.0130,1.2870


In [64]:
### calculate incremental rides
new_pp_rides['pp_counterfactual_rides'] = ((new_pp_rides['num_rides_new_pp']*(1-new_pp_rides['perc_addl_activations']))).round(0)
new_pp_rides['pp_addl_rides'] = new_pp_rides['num_rides_new_pp']- new_pp_rides['pp_counterfactual_rides'] 
new_pp_rides['pp_incre_rides_from_counterfactual'] = ((1-1/(1+new_pp_rides['pre_existing_rider_lift']))*new_pp_rides['pp_counterfactual_rides']).round(0)
new_pp_rides['pp_incre_rides_from_addl'] = ((1-1/(1+new_pp_rides['new_rider_pp_lift']))*new_pp_rides['pp_addl_rides']).round(0)

In [65]:
new_pp_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_pp,cum_rides_new_pp,week,perc_addl_activations,pre_existing_rider_lift,new_rider_pp_lift,pp_counterfactual_rides,pp_addl_rides,pp_incre_rides_from_counterfactual,pp_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user pp,2506,2506,2025-08-04,0.000000,0.0227,1.8330,2506.0,0.0,56.0,0.0
1,33,2025,2025-08-11,1,new user pp,3119,5625,2025-08-11,0.000000,0.0227,1.8330,3119.0,0.0,69.0,0.0
2,34,2025,2025-08-18,2,new user pp,3779,9404,2025-08-18,0.053942,0.0227,1.8330,3575.0,204.0,79.0,132.0
3,35,2025,2025-08-25,3,new user pp,4116,13520,2025-08-25,0.000000,0.0227,1.8330,4116.0,0.0,91.0,0.0
4,36,2025,2025-09-01,4,new user pp,4043,17563,2025-09-01,0.000000,0.0227,1.8330,4043.0,0.0,90.0,0.0
5,37,2025,2025-09-08,5,new user pp,6330,23893,2025-09-08,0.094709,0.0227,1.8330,5730.0,600.0,127.0,388.0
6,38,2025,2025-09-15,6,new user pp,7771,31664,2025-09-15,0.116335,0.0227,1.8330,6867.0,904.0,152.0,585.0
7,39,2025,2025-09-22,7,new user pp,7516,39180,2025-09-22,0.079330,0.0227,1.8330,6920.0,596.0,154.0,386.0
8,40,2025,2025-09-29,8,new user pp,7682,46862,2025-09-29,0.093079,0.0227,1.8330,6967.0,715.0,155.0,463.0
9,41,2025,2025-10-06,9,new user pp,9508,56370,2025-10-06,0.169360,0.0130,1.2870,7898.0,1610.0,101.0,906.0


#### get new user np data

In [66]:
new_user_np_sql=f"""
WITH week_bounds AS (
  SELECT 
    week_start,
    DATE_ADD('day', 6, week_start) AS week_end
  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'{report_end_dt}', INTERVAL '7' DAY)) AS t(week_start)
)

SELECT
  week(wb.week_start) as week_num,
  year(wb.week_start) as year_num,
  wb.week_start as starting_week_date,
  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,
  'new user np' as user_type,
  COUNT(DISTINCT ride_id) AS num_rides_new_np,
  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np
FROM week_bounds wb
LEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d
  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)
  AND d.user_type = 'new-user-NP'
  AND d.most_used_business_program_type_cleaned = 'managed'
  AND d.most_used_bp_activated_at BETWEEN DATE'2025-08-04' AND date(wb.week_end)
GROUP BY 1,2,3,4,5
ORDER BY 3
"""
new_np_rides = pd.DataFrame()
new_np_rides = trino.query(new_user_np_sql,scheduled =True)
new_np_rides

{"ts": "2025-11-18T00:15:51.964Z", "uuid": "fb16dcd5-5dfd-4780-aad6-ceaf7fb47f99", "name": "lyft_data_toolkit.v1_to_v3.trino", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/trino.py:26", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: trino.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}


Created Mozart command
{'mozart_id': 569120568, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569120568', 'query': "WITH week_bounds AS (\n  SELECT \n    week_start,\n    DATE_ADD('day', 6, week_start) AS week_end\n  FROM UNNEST(SEQUENCE(DATE '2025-08-04', DATE'2025-11-16', INTERVAL '7' DAY)) AS t(week_start)\n)\n\nSELECT\n  week(wb.week_start) as week_num,\n  year(wb.week_start) as year_num,\n  wb.week_start as starting_week_date,\n  date_diff('week',date'2025-08-04', week_start) as post_reward_week_num,\n  'new user np' as user_type,\n  COUNT(DISTINCT ride_id) AS num_rides_new_np,\n  sum(count(distinct ride_id)) over (order by date_diff('week',date'2025-08-04', week_start) asc) as cum_rides_new_np\nFROM week_bounds wb\nLEFT JOIN shuninga.causal_analysis_bp_raw_data_step_4 d\n  ON DATE(d.requested_at) BETWEEN date(wb.week_start) AND date(wb.week_end)\n  AND d.user_type = 'new-user-NP'\n  AND d.most_used_business_program_type_cl

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np
0,32,2025,2025-08-04,0,new user np,173,173
1,33,2025,2025-08-11,1,new user np,356,529
2,34,2025,2025-08-18,2,new user np,480,1009
3,35,2025,2025-08-25,3,new user np,478,1487
4,36,2025,2025-09-01,4,new user np,552,2039
5,37,2025,2025-09-08,5,new user np,741,2780
6,38,2025,2025-09-15,6,new user np,937,3717
7,39,2025,2025-09-22,7,new user np,945,4662
8,40,2025,2025-09-29,8,new user np,957,5619
9,41,2025,2025-10-06,9,new user np,1226,6845


In [67]:
new_np_rides = pd.merge(new_np_rides,
                        np_preds[["week","perc_addl_activations"]],
                        left_on = "starting_week_date",
                        right_on = "week",
                        how = "left"
                       ) 
new_np_rides = pd.merge(new_np_rides,
                        lift_lookup[['starting_week_date','pre_existing_rider_lift']],
                        on = 'starting_week_date',
                        how = 'left')

In [68]:
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.007463,0.0227
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0227
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0227
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0227
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0227
5,37,2025,2025-09-08,5,new user np,741,2780,2025-09-08,0.000000,0.0227
6,38,2025,2025-09-15,6,new user np,937,3717,2025-09-15,0.063953,0.0227
7,39,2025,2025-09-22,7,new user np,945,4662,2025-09-22,0.046358,0.0227
8,40,2025,2025-09-29,8,new user np,957,5619,2025-09-29,0.258824,0.0227
9,41,2025,2025-10-06,9,new user np,1226,6845,2025-10-06,0.220513,0.0130


In [69]:
### calculate incremental rides
new_np_rides['np_counterfactual_rides'] = (new_np_rides['num_rides_new_np']*(1-new_np_rides['perc_addl_activations'])).round(0)
new_np_rides['np_addl_rides'] = new_np_rides['num_rides_new_np']- new_np_rides['np_counterfactual_rides'] 
new_np_rides['np_incre_rides_from_counterfactual'] = ((1-1/(1+new_np_rides['pre_existing_rider_lift']))*new_np_rides['np_counterfactual_rides']).round(0)
new_np_rides['np_incre_rides_from_addl'] = new_np_rides['np_addl_rides']
new_np_rides

,week_num,year_num,starting_week_date,post_reward_week_num,user_type,num_rides_new_np,cum_rides_new_np,week,perc_addl_activations,pre_existing_rider_lift,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl
0,32,2025,2025-08-04,0,new user np,173,173,2025-08-04,0.007463,0.0227,172.0,1.0,4.0,1.0
1,33,2025,2025-08-11,1,new user np,356,529,2025-08-11,0.000000,0.0227,356.0,0.0,8.0,0.0
2,34,2025,2025-08-18,2,new user np,480,1009,2025-08-18,0.000000,0.0227,480.0,0.0,11.0,0.0
3,35,2025,2025-08-25,3,new user np,478,1487,2025-08-25,0.000000,0.0227,478.0,0.0,11.0,0.0
4,36,2025,2025-09-01,4,new user np,552,2039,2025-09-01,0.000000,0.0227,552.0,0.0,12.0,0.0
5,37,2025,2025-09-08,5,new user np,741,2780,2025-09-08,0.000000,0.0227,741.0,0.0,16.0,0.0
6,38,2025,2025-09-15,6,new user np,937,3717,2025-09-15,0.063953,0.0227,877.0,60.0,19.0,60.0
7,39,2025,2025-09-22,7,new user np,945,4662,2025-09-22,0.046358,0.0227,901.0,44.0,20.0,44.0
8,40,2025,2025-09-29,8,new user np,957,5619,2025-09-29,0.258824,0.0227,709.0,248.0,16.0,248.0
9,41,2025,2025-10-06,9,new user np,1226,6845,2025-10-06,0.220513,0.0130,956.0,270.0,12.0,270.0


In [70]:
pre_exisiting_rides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   week_num                 15 non-null     int64         
 1   year_num                 15 non-null     int64         
 2   starting_week_date       15 non-null     datetime64[ns]
 3   post_reward_week_num     15 non-null     int64         
 4   user_type                15 non-null     object        
 5   num_rides_pre_existing   15 non-null     int64         
 6   cum_rides_pre_existing   15 non-null     int64         
 7   pre_existing_rider_lift  14 non-null     float64       
 8   pre_existing_incr_rides  14 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(1)
memory usage: 1.2+ KB


In [71]:
#### get final result with all incrementality

incremental_all_rides = pd.merge(
                        pre_exisiting_rides[['week_num','year_num','starting_week_date','post_reward_week_num','num_rides_pre_existing','cum_rides_pre_existing','pre_existing_rider_lift','pre_existing_incr_rides']],
                        new_pp_rides[['starting_week_date','num_rides_new_pp','cum_rides_new_pp','new_rider_pp_lift','pp_counterfactual_rides','pp_addl_rides','pp_incre_rides_from_counterfactual','pp_incre_rides_from_addl']],
                        on = 'starting_week_date',
                        how = 'left'
)

incremental_all_rides = pd.merge(
    incremental_all_rides,
    new_np_rides[['starting_week_date','num_rides_new_np','cum_rides_new_np','np_counterfactual_rides','np_addl_rides','np_incre_rides_from_counterfactual','np_incre_rides_from_addl']],
    on = 'starting_week_date',
    how = 'left'
)
incremental_all_rides['total_incre_rides']= incremental_all_rides['pre_existing_incr_rides']+ incremental_all_rides['pp_incre_rides_from_counterfactual']+ incremental_all_rides['pp_incre_rides_from_addl']+incremental_all_rides['np_incre_rides_from_counterfactual']+incremental_all_rides['np_incre_rides_from_addl']
incremental_all_rides['cum_total_incre_rides'] = incremental_all_rides['total_incre_rides'].cumsum()
incremental_all_rides['total_observed_rides'] = incremental_all_rides['num_rides_pre_existing']+incremental_all_rides['num_rides_new_pp']+incremental_all_rides['num_rides_new_np']
incremental_all_rides['cum_total_observed_rides'] = incremental_all_rides['total_observed_rides'].cumsum()
incremental_all_rides['cum_baseline_rides'] = incremental_all_rides['cum_total_observed_rides']-incremental_all_rides['cum_total_incre_rides']
incremental_all_rides['progress_t_target'] = incremental_all_rides['cum_total_incre_rides']/713894
incremental_all_rides

,week_num,year_num,starting_week_date,post_reward_week_num,num_rides_pre_existing,cum_rides_pre_existing,pre_existing_rider_lift,pre_existing_incr_rides,num_rides_new_pp,cum_rides_new_pp,...,np_counterfactual_rides,np_addl_rides,np_incre_rides_from_counterfactual,np_incre_rides_from_addl,total_incre_rides,cum_total_incre_rides,total_observed_rides,cum_total_observed_rides,cum_baseline_rides,progress_t_target
0,32,2025,2025-08-04,0,71479,71479,0.0227,1587.0,2506,2506,...,172.0,1.0,4.0,1.0,1648.0,1648.0,74158,74158,72510.0,0.002308
1,33,2025,2025-08-11,1,70331,141810,0.0227,1561.0,3119,5625,...,356.0,0.0,8.0,0.0,1638.0,3286.0,73806,147964,144678.0,0.004603
2,34,2025,2025-08-18,2,70152,211962,0.0227,1557.0,3779,9404,...,480.0,0.0,11.0,0.0,1779.0,5065.0,74411,222375,217310.0,0.007095
3,35,2025,2025-08-25,3,67829,279791,0.0227,1506.0,4116,13520,...,478.0,0.0,11.0,0.0,1608.0,6673.0,72423,294798,288125.0,0.009347
4,36,2025,2025-09-01,4,64151,343942,0.0227,1424.0,4043,17563,...,552.0,0.0,12.0,0.0,1526.0,8199.0,68746,363544,355345.0,0.011485
5,37,2025,2025-09-08,5,78855,422797,0.0227,1750.0,6330,23893,...,741.0,0.0,16.0,0.0,2281.0,10480.0,85926,449470,438990.0,0.014680
6,38,2025,2025-09-15,6,81989,504786,0.0227,1820.0,7771,31664,...,877.0,60.0,19.0,60.0,2636.0,13116.0,90697,540167,527051.0,0.018372
7,39,2025,2025-09-22,7,77048,581834,0.0227,1710.0,7516,39180,...,901.0,44.0,20.0,44.0,2314.0,15430.0,85509,625676,610246.0,0.021614
8,40,2025,2025-09-29,8,74461,656295,0.0227,1653.0,7682,46862,...,709.0,248.0,16.0,248.0,2535.0,17965.0,83100,708776,690811.0,0.025165
9,41,2025,2025-10-06,9,79833,736128,0.0130,1025.0,9508,56370,...,956.0,270.0,12.0,270.0,2314.0,20279.0,90567,799343,779064.0,0.028406


In [74]:
incremental_all_rides = incremental_all_rides[incremental_all_rides['week_num']!=46]

In [75]:
incremental_all_rides.to_csv('incremtal_all_rides.csv',index= False)

In [ ]:
incremental_all_rides.to_hive_table(table_name='bp_incremental_all_rides', schema_name='shuninga', if_exists = 'replace')

{"ts": "2025-11-18T00:20:09.781Z", "uuid": "4c67501e-cf66-471e-90cf-d7e2eb26d33a", "name": "lyft_data_toolkit.v1_to_v3.hive", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/hive.py:32", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: to_hive_table method is outdated.Please use the new write_to_table function in the lyft_data_toolkit package."}
{"ts": "2025-11-18T00:20:10.143Z", "uuid": "89fc074d-c013-483d-8a8e-1a3c13efb819", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "b8758af9c9", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 569123460, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569123460', 'query': "SHOW TABLES FROM shuninga LIKE 'bp_incremental_all_rides'"}


{"ts": "2025-11-18T00:22:11.007Z", "uuid": "8b5a2511-7bfd-478d-adfc-c98a191fdcec", "name": "lyft_data_toolkit.db.read._read.Query", "lvlname": "WARNING", "lvlno": 30, "pid": 122, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/db/read/_read.py:232", "revision": "b8758af9c9", "msg": "execution_context not supported for spark"}


Created Mozart command
{'mozart_id': 569124053, 'backend': 'spark', 'native_command_id': 'pending', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/569124053', 'query': 'DROP TABLE IF EXISTS shuninga.bp_incremental_all_rides'}


### extra cells 

In [ ]:
# del lift_lookup

In [ ]:
# columns = [
#     "week_num", # begining week number of biweekly              
#     "year_num",  # beginning year number of the biweekly period             
#     "starting_week_date",    
#     "pre_existing_rider_lift", # get it from the tool
#     "new_rider_pp_lift"
# ]

# lift_lookup = pd.DataFrame(columns=columns)

# #  Function to append new biweekly data
# def add_biweekly_entry(df, week_num, year_num, starting_week_date, pre_existing_rider_lift, new_rider_pp_lift):
#     new_row = {
#         "week_num": int(week_num),
#         "year_num": int(year_num),
#         "starting_week_date": pd.to_datetime(starting_week_date),
#         "pre_existing_rider_lift": float(pre_existing_rider_lift),
#         "new_rider_pp_lift": float(new_rider_pp_lift)
#     }
#     return df.append(new_row, ignore_index=True)

# # do this biweekly to update the dataframe
# lift_lookup = add_biweekly_entry(lift_lookup, week_num=44, year_num=2025, starting_week_date="2025-11-02", pre_existing_rider_lift= 0.6, new_rider_pp_lift=1)

# # optional
# lift_lookup.to_csv("biweekly_lift_data.csv", index=False)

# lift_lookup